### <center size=2> Real data example : Reconcavo Basin, Brazil <center> 
    

--------------------------------------------------------------------------------------------------------------------

In [ ]:
#----------------------------- ALL INTERNAL PACKAGES --------------------------- #
import numpy as np
import ipywidgets as widgets
import scipy as sp
import pandas as pd
import pylab as py
import matplotlib.pyplot as plt
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from IPython.display import Latex
#from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import axes3d
#import pyproj
import utm

# <center> Etapa 01: Leitura dos dados de gravidade <center>

In [ ]:
# ------------- Use pandas to read the real data -----------------:
# dados parcialmente interpolados na regiao do rifte abortado (RECOMENDACAO: LER LAT, LON): 
nome=['Lat','Lon', 'UTMe', 'UTMn', 'Boug', 'Elev']
data = pd.read_csv('GRAV_DEXBADEXES_med_proc.asc', sep=',', header=20, usecols=(2,3,4,5,7,12), names=nome, na_values= 9999.99 )

#name=['Lon','Lat', 'Boug']
#bndg= pd.read_csv('dados_bndg.asc', sep=',', header=17, usecols=(0,1,13), names=name, na_values= '*' )
#bndg

In [ ]:
# ------------- Use pandas to read the real data -----------------:
# dados parcialmente interpolados na regiao do rifte abortado (RECOMENDACAO: LER LAT, LON): 
nome=['UTMe', 'UTMn', 'Line', 'DZ',]
data = pd.read_csv('BOUG_RESIDUAL_Cont4000.xyz', sep='\s+', header=0, names=nome, na_values= 9999.99 )

#name=['Lon','Lat', 'Boug']
#bndg= pd.read_csv('dados_bndg.asc', sep=',', header=17, usecols=(0,1,13), names=name, na_values= '*' )
data

## 1.1: Leitura dos dados do ICGEM para a anomalia bouguer regional

In [ ]:
# dados parcialmente interpolados na regiao do rifte abortado (RECOMENDACAO: LER LAT, LON): 
nome=['Lon','Lat', 'Reg_Boug']
regional = pd.read_csv('Regional.dat', header=37, sep='\s+', usecols=(1,2,4), names=nome )

In [ ]:
regional

# <center> Etapa 02: Leitura das coordenadas dos dados de poço <center>

In [ ]:
# Leitura das coordenadas utm dos poços da bacia do reconcavo:
# dados parcialmente interpolados na regiao do rifte abortado (RECOMENDACAO: LER LAT, LON): 
label=['ID','Nome', 'UTMn', 'UTMe']
poco = pd.read_csv('INFO_POCOS2.txt', sep='\s+', header=0, usecols=(0,1,2,3), names=label, na_values=999.999 ) # 999.999 = dummy

In [ ]:
# lat lon dos poços computados pela funcao utm:
npo = len(poco)
print ('numero de pocos=', npo)
lon_poc = np.zeros(npo)
lat_poc = np.zeros(npo)

for i in range(npo):
    lat_poc[i], lon_poc[i] = utm.to_latlon(poco['UTMe'][i], poco['UTMn'][i],zone_number=24, zone_letter='K' )

# adicionando as colunas 'Lat' e 'Lon' no dataframe poco:
poco['Lon'] = lon_poc
poco['Lat'] = lat_poc    


# <center> Etapa 03: Leitura das coordenadas das linhas sismicas <center>

In [ ]:
# Leitura das coordenadas utm de algumas linhas sismicas da bacia do reconcavo:
#line = ['linha', 'w0', 's0', 'wf', 'sf']
#sis = pd.read_csv('loc_sismica.txt', sep='\s+', header=0 ) # 999.999 = dummy
sis = pd.read_csv('NOVO_INFO_SISMICA.csv', sep=',', header=0 ) # 999.999 = dummy

In [ ]:
# conversao para grau decimal (dados em grau,minuto,segundo):
lon0 = -(sis['wg0'] + sis['wm0']/60.0 + sis['ws0']/3600.0)
lat0 = -(sis['sg0'] + sis['sm0']/60.0 + sis['ss0']/3600.0)

lonf = -(sis['wgf'] + sis['wmf']/60.0 + sis['wsf']/3600.0)
latf = -(sis['sgf'] + sis['smf']/60.0 + sis['ssf']/3600.0)
lat0
sis

In [ ]:
# plot das linhas sismicas lidas do arquivo:
nlines = len(lat0)
ys=np.zeros(3)
xs=np.zeros (3)
for i in range(0, nlines, 1):
    ys = [ lat0[i], latf[i] ]
    xs = [ lon0[i], lonf[i] ]
    plt.plot(xs, ys,'.-')
plt.show()
np.array(xs)
np.array(ys)
xs

# <center> Etapa 04: Visualização dos dados : <center>

In [ ]:
# ------------ plot the flight lines to check for the interpolation:--------#
# ------------ set figure size: ------------------------------------------#
plt.close('all')
fig = plt.figure(figsize=(20,12), facecolor='w')
plt.subplot(121)
#plt.title('Dado de Gravidade - Bacia do Reconcavo',fontsize=15)
plt.xlabel('Longitude($^o$)', fontsize = 15)
plt.ylabel('Latitude($^o$)', fontsize = 15)
p1 = plt.scatter(data['Lon'], data['Lat'], s=1.0, c=data['Boug'], marker='.', cmap=plt.cm.jet)
cbar = plt.colorbar(p1, aspect=30, fraction = 0.03, orientation="horizontal")
cbar.set_label('Anomalia Bouguer Total $ (mGal) $',fontsize=15, rotation = 0)
plt.grid()
#plt.savefig('figs finais/Python/anomalia_total.png', dpi=300, transparent=True, bbox_inches="tight" )

#################################################################################
plt.subplot(122)
#plt.title('Dado de Gravidade - Bacia do Reconcavo',fontsize=15)
plt.xlabel('Longitude($^o$)', fontsize = 15)
plt.ylabel('Latitude($^o$)', fontsize = 15)
p2 = plt.scatter(regional['Lon'], regional['Lat'], s=1.0, c=regional['Reg_Boug'], marker='.', cmap=plt.cm.jet)
cbar = plt.colorbar(p2, aspect=30, fraction = 0.03, orientation="horizontal")
cbar.set_label('Anomalia Bouguer bndg $ (mGal) $',fontsize=15, rotation = 0)
plt.grid()
#plt.savefig('figs finais/Python/anomalia_total.png', dpi=300, transparent=True, bbox_inches="tight" )

plt.grid()
#plt.savefig('Separacao.png', dpi=300, transparent=True, bbox_inches="tight" )


plt.show()

#  <center size=3> Etapa 06: Remoção do Regional da Anomalia Bouguer: <center>

## 6.1: via ajuste polinomial

In [ ]:
# create a polinomial data:
xp   = data['Lon']
yp   = data['Lat']
bou  = data['Boug']
ndat = len(xp)
one  = np.ones(ndat)

In [ ]:
# utilizando um polinomio de grau 6:
S = np.array( [ one, xp, yp, xp**2, yp**2, xp**3, yp**3, xp**4, yp**4, xp**5, yp**5, xp**6, yp**6])
#print (type(S))
# transposta para arrumar a matrix corretamente:
A = S.T

In [ ]:
# estimativa de minimos quadrados para obter os coeficientes do polinomio:
p = np.linalg.solve(np.dot(A.T,A) , np.dot(A.T,bou) )
print (p)

In [ ]:
# calculo dos dados preditos pelo modelo polinomial:
bou_pred = np.dot(A,p)

In [ ]:
# calculo da anomalia bouguer residual via modelo polinomial:
residual = bou - bou_pred
# incorporar ao dataframe:
data['Boug_residual'] = residual
data['Boug_regional'] = bou_pred

In [ ]:
data

##  6.2:  via ICGEM (dados de satélite)

In [ ]:
Bou_res = data.Boug - regional.Reg_Boug
data['Boug_res'] = np.array(Bou_res)

In [ ]:
data

##  6.3:  via polinomial robusto 

In [ ]:
# Uso do polinomio simples como entrada
p2 = p.copy()
bou_pred2 = bou_pred.copy()

In [ ]:
# Quantidade de iteracoes
iteracoes = 30

In [ ]:
# Calculo do polinomio robusto
for i in range(iteracoes):
    r = bou - bou_pred2
    #s = np.median(r)
    #W = np.diag(1./np.abs(r + 1.e-10))
    #W = np.dot(A.T,W)
    #p2 = np.linalg.solve(np.dot(W,A), np.dot(W,bou))
    p2=np.linalg.solve(np.dot(A.T,A),np.dot(A.T,r))
    bou_pred2 = np.dot(A, p2)

In [ ]:
residual2= bou - bou_pred2
data['Boug_residual_robusto'] = residual2

In [ ]:
data

## <center size=3> Etapa 06: Visualização da anomalia Bouguer regional e residual: <center>

## 6.1: Via ajuste polinomial

In [ ]:
# Plot residual bouguer anomaly:
plt.close('all')

plt.figure(figsize=(3.5,12), facecolor='w')

#plt.title(' Anomalia Bouguer Regional ',fontsize=15)
plt.xlabel('Longitude($^o$)', fontsize = 15)
plt.ylabel('Latitude ($^o$)', fontsize = 15)
p1 = plt.scatter(data['Lon'], data['Lat'], s=60.0, c=data['Boug_regional'], marker='.', cmap=plt.cm.jet)
cbar =plt.colorbar(p1, aspect=30, fraction = 0.03, orientation="horizontal")
cbar.set_label('$ (mGal) $',fontsize=15, rotation = 0)
plt.grid()
#plt.savefig('figs finais/Python/anom_regional2.png', dpi=300, transparent=True, bbox_inches="tight" )

plt.show()
###########################################################################################################################


In [ ]:
#plt.title(' Anomalia Bouguer Residual',fontsize=15)
plt.figure(figsize= (3.5,12), facecolor='w')
plt.xlabel('Longitude ($^o$)', fontsize = 15)
p2 = plt.scatter(data['Lon'], data['Lat'], s=60.0, c=data['Boug_residual'], marker='.', cmap=plt.cm.jet, )
cbar = plt.colorbar(p2, aspect=30, fraction = 0.03, orientation="horizontal")
cbar.set_label('$ (mGal) $',fontsize=15,rotation = 0)
plt.grid()
#plt.savefig('figs finais/Python/anom_residual2.png', dpi=300, transparent=True, bbox_inches="tight" )

plt.show()

## 6.2: Via ajuste polinomial robusto

In [ ]:
# Plot residual bouguer anomaly:
plt.close('all')

plt.figure(figsize=(3.5,12), facecolor='w')

#plt.title(' Anomalia Bouguer Regional ',fontsize=15)
plt.xlabel('Longitude($^o$)', fontsize = 15)
plt.ylabel('Latitude ($^o$)', fontsize = 15)
p1 = plt.scatter(data['Lon'], data['Lat'], s=60.0, c=residual2, marker='.', cmap=plt.cm.jet)
cbar =plt.colorbar(p1, aspect=30, fraction = 0.03, orientation="horizontal")
cbar.set_label('$ (mGal) $',fontsize=15, rotation = 0)
plt.grid()
#plt.savefig('figs finais/Python/anom_residual_robusto6.png', dpi=300, transparent=True, bbox_inches="tight" )

plt.show()
###########################################################################################################################

## Delimitação da área à Bacia do Recôncavo

In [ ]:
# Dado grav:
# longitude filter:
reconcavo = data.loc[ (data['Lon']<= -38.00) & (data['Lon']>= -39.0) ] # should have the whitespace " LONG" !
# latitude filter:
rec = reconcavo.loc[ (reconcavo['Lat']<= -12.0) & (reconcavo['Lat']>= -13.00) ]
# check if is right:
rec.max(), rec.min()

## Comparação dos dados BDEP cm dados BNDG

In [ ]:
# ------------ plot the flight lines to check for the interpolation:--------#
# ------------ set figure size: ------------------------------------------#
plt.close('all')
fig = plt.figure(figsize=(20,12), facecolor='w')
plt.subplot(121)
#plt.title('Dado de Gravidade - Bacia do Reconcavo',fontsize=15)
plt.xlabel('Longitude($^o$)', fontsize = 15)
plt.ylabel('Latitude($^o$)', fontsize = 15)
p1 = plt.scatter(rec['Lon'], rec['Lat'], s=1.0, c=rec['Boug'], marker='.', cmap=plt.cm.jet)
cbar = plt.colorbar(p1, aspect=30, fraction = 0.03, orientation="horizontal")
cbar.set_label('Anomalia Bouguer Total $ (mGal) $',fontsize=15, rotation = 0)
plt.grid()
#plt.savefig('figs finais/Python/anomalia_total.png', dpi=300, transparent=True, bbox_inches="tight" )

#################################################################################
plt.subplot(122)
#plt.title('Dado de Gravidade - Bacia do Reconcavo',fontsize=15)
plt.xlabel('Longitude($^o$)', fontsize = 15)
plt.ylabel('Latitude($^o$)', fontsize = 15)
p2 = plt.scatter(bndg['Lon'], bndg['Lat'], s=10.0, c=bndg['Boug'], marker='.', cmap=plt.cm.jet)
cbar = plt.colorbar(p2, aspect=30, fraction = 0.03, orientation="horizontal")
cbar.set_label('Anomalia Bouguer BNDG $ (mGal) $',fontsize=15, rotation = 0)
plt.grid()
#plt.savefig('figs finais/Python/anomalia_total.png', dpi=300, transparent=True, bbox_inches="tight" )
#plt.savefig('Separacao.png', dpi=300, transparent=True, bbox_inches="tight" )


plt.show()

In [ ]:
# ------------------------- Define the center of the map -----------------------------------#  
lo0 = 0.5*( rec['Lon'].max() + rec['Lon'].min() )
la0 = 0.5*( rec['Lat'].max() + rec['Lat'].max() )

# ------- use basemap function to create the plot : ---------# 
m = Basemap(llcrnrlon=-39.0, llcrnrlat=-13.0,
            urcrnrlon=-38.0, urcrnrlat=-12.0, 
            resolution='l',projection='tmerc', 
            lon_0=lo0, lat_0=la0,epsg=29100)

In [ ]:
lons, lats = m( np.array( rec['Lon'] ), np.array( rec['Lat']) )

In [ ]:
# ------------ set figure size: ------------------------------------------#
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(15,15), facecolor='w')

#----------Drawing Lat Long: -------------#
m.drawmeridians( range( 0,360,1), labels=[0, 1, 0, 1], latmax=rec['Lat'].min() )
m.drawparallels( range(-90,90,1), labels=[1, 0, 1, 0], latmax=rec['Lat'].max() )
m.drawmapscale(-38.1, -12.85, 3, 3, 10.0, units='km', barstyle='simple', fontcolor='k')

#------------------Loading Shapefiles---------------#
#m.readshapefile('brazil_shapefile/BRA_adm0', 'Brasil', drawbounds=True, linewidth=1)
#m.readshapefile('Real/shapes/MatoGrossodoSul', 'MatoGrossodoSul', drawbounds=True, linewidth=1)
#m.readshapefile('Real/shapes/Parana', 'Parana', drawbounds=True, linewidth=1)
#m.readshapefile('shapeFiles/bacias/Limite_das_Bacias_Sedimentares', 'Limite_das_Bacias_Sedimentares', 
            #    drawbounds=True, linewidth=1)
#m.readshapefile('shapeFiles/CAMPOS_PRODUCAO_191118', 'CAMPOS_PRODUCAO_191118', drawbounds=True, linewidth=0.5)


#--------------Background satellite image options-----------#

#m.shadedrelief(scale=1)
#m.etopo(scale=2)
#m.bluemarble(scale=0.5)
#m.arcgisimage(service='ESRI_Imagery_World_2D', xpixels=1000, ypixels=1000, verbose=True)
m.arcgisimage(service='World_Shaded_Relief', xpixels=1000, ypixels=1000, verbose= True)


#------------------Mapa da Anomalia------------------#
k = m.scatter(lons, lats, s=5.0, c=rec['Boug_residual'], marker='o', cmap=plt.cm.jet, latlon=False)
#----- colorbar specifications -----:
cbar = m.colorbar(k, location='right', size="5%", pad="5%")
cbar.set_label('Anomalia Bouguer Residual $ (mGal) $',fontsize=13, rotation=90)
#plt.savefig('figs finais/Python/mapa_bacia_residual.png', dpi=300, transparent=True, bbox_inches="tight" )


## 6.2: via ICGEM

In [ ]:
# Dado grav:
# longitude filter:
reconcavo = data.loc[ (data['Lon']<= -38.00) & (data['Lon']>= -39.0) ] # should have the whitespace " LONG" !
# latitude filter:
rec = reconcavo.loc[ (reconcavo['Lat']<= -12.0) & (reconcavo['Lat']>= -13.00) ]
# check if is right:
rec.max(), rec.min()

In [ ]:
# ------------------------- Define the center of the map -----------------------------------#  
lo0 = 0.5*( rec['Lon'].max() + rec['Lon'].min() )
la0 = 0.5*( rec['Lat'].max() + rec['Lat'].max() )

# ------- use basemap function to create the plot : ---------# 
m = Basemap(llcrnrlon=-39.0, llcrnrlat=-13.0,
            urcrnrlon=-38.0, urcrnrlat=-12.0, 
            resolution='l',projection='tmerc', 
            lon_0=lo0, lat_0=la0,epsg=29100)

In [ ]:
# referenciando as coordenadas grav, sismica e poco ao mapa:
lons, lats = m( np.array( rec['Lon'] ), np.array( rec['Lat']) )

In [ ]:
# ------------ set figure size: ------------------------------------------#
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(15,15), facecolor='w')

#----------Drawing Lat Long: -------------#
m.drawmeridians( range( 0,360,1), labels=[0, 1, 0, 1], latmax=rec['Lat'].min() )
m.drawparallels( range(-90,90,1), labels=[1, 0, 1, 0], latmax=rec['Lat'].max() )
m.drawmapscale(-38.1, -12.85, 3, 3, 10.0, units='km', barstyle='simple', fontcolor='k')

#------------------Loading Shapefiles---------------#
#m.readshapefile('brazil_shapefile/BRA_adm0', 'Brasil', drawbounds=True, linewidth=1)
#m.readshapefile('Real/shapes/MatoGrossodoSul', 'MatoGrossodoSul', drawbounds=True, linewidth=1)
#m.readshapefile('Real/shapes/Parana', 'Parana', drawbounds=True, linewidth=1)
#m.readshapefile('shapeFiles/bacias/Limite_das_Bacias_Sedimentares', 'Limite_das_Bacias_Sedimentares', 
              #  drawbounds=True, linewidth=1)
#m.readshapefile('shapeFiles/CAMPOS_PRODUCAO_191118', 'CAMPOS_PRODUCAO_191118', drawbounds=True, linewidth=0.5)


#--------------Background satellite image options-----------#

#m.shadedrelief(scale=1)
#m.etopo(scale=2)
#m.bluemarble(scale=0.5)
#m.arcgisimage(service='ESRI_Imagery_World_2D', xpixels=1000, ypixels=1000, verbose=True)
m.arcgisimage(service='World_Shaded_Relief', xpixels=1000, ypixels=1000, verbose= True)



#------------------Mapa da Anomalia------------------#
k = m.scatter(lons, lats, s=5.0, c=rec['Boug_res'], marker='o', cmap=plt.cm.jet, latlon=False)
#----- colorbar specifications -----:
cbar = m.colorbar(k, location='right', size="2%", pad="2%")
cbar.set_label('Anomalia Bouguer Total $ (mGal) $',fontsize=20, rotation=90)
#plt.savefig('figs finais/Python/mapa_bacia.png', dpi=300, transparent=True, bbox_inches="tight" )

#  <center> Etapa 07: Delimitação da área de estudo ao compartimento sul:

In [ ]:
# Dado grav:
# longitude filter:
target = data.loc[ (data['Lon']<= -38.00) & (data['Lon']>= -39.2) ] # should have the whitespace " LONG" !
# latitude filter:
target = target.loc[ (target['Lat']<= -12.5) & (target['Lat']>= -12.75) ]
# check if is right:
target.max(), target.min()

In [ ]:
# Dado poco:
# longitude filter:
poco = poco.loc[ (poco['Lon']<= -38.4) & (poco['Lon']>= -38.90) ] # should have the whitespace " LONG" !
# latitude filter:
poco = poco.loc[ (poco['Lat']<= -12.50) & (poco['Lat']>= -12.7) ]
# check if is right:
poco.max(), poco.min()

In [ ]:
ipoco = poco.index.tolist()
print (ipoco[0])

## <center size=3> Etapa 08: Plot da Anomalia residual, as linhas sismicas e os poços: <center>

In [ ]:

# ------------ set figure size: ------------------------------------------#
plt.close('all')
fig = plt.figure(figsize=(12,4), facecolor='w')

plt.xlabel('Longitude ($^o$)', fontsize = 15)
plt.ylabel('Latitude ($^o$)', fontsize = 15)
p1 = plt.scatter(target['Lon'], target['Lat'], s=60.0, c=target['Boug_residual'], marker='.', cmap=plt.cm.jet)
cbar = plt.colorbar(p1, aspect=30, fraction = 0.03, orientation="vertical")
cbar.set_label('Anomalia Bouguer Residual $ (mGal) $',fontsize=15, rotation = 90)

# plot das linhas sismicas:
for i in range(0,2,1):
    ys[i] = [lat0[i], latf[i]]
    xs[i] = [lon0[i], lonf[i]]
    plt.plot(xs[i], ys[i], marker='.', linestyle='solid', linewidth=3.0, markersize=0, label=sis['LINHA'][i] )    

#plot dos pocos:
for k in range(0, len(poco), 1 ):
    i = ipoco[k] 
    plt.plot(poco['Lon'][i], poco['Lat'][i], marker='*', linestyle='None', linewidth=3.0, markersize=15, label=poco['Nome'][i] )

plt.legend(loc=2)

###########################################################################################################################
#plt.savefig('figs finais/Python/area_estudo_multigeofisico.png', dpi=300, transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
np.shape(ys)

## <center> Etapa 9: Definicao do perfil gravimétrico a partir do banco de dados pre-filtrados: <center>

In [ ]:
# bibliotecas externas:
from modules.grids import profile

In [ ]:
# construcao dos pontos inicial e final do perfil em funcao da linha sismica:


p1=(-38.87, -12.55) #ponto na borda flexural da bacia (VAI FAZER A BACIA AFLORAR EM Z=0)

p2= (-38.18, -12.675) #ponto na borda de falha da bacia 


In [ ]:
# calculo do tamanho do perfil

p1_m = utm.from_latlon(-38.87,-12.55, force_zone_number=24)
p2_m= utm.from_latlon(-38.18,-12.675, force_zone_number=24)

dist= (((p1_m[0]-p2_m[0])**2 +(p1_m[1]-p2_m[1])**2)**(0.5))/1000 #distancia em Km
#dist
p1_m
p2_m

In [ ]:
size = int(np.size(target['Boug']) * 10.0/100.0 )
size

In [ ]:
perfil = profile(target['Lon'], target['Lat'], target['Boug_residual'], p1, p2, size)

In [ ]:
perfil_robusto=profile(target['Lon'], target['Lat'], target['Boug_residual_robusto'], p1, p2, size)

In [ ]:
# calculando a conversao dos dados do perfil para utm:
N_perf = np.zeros(size)
E_perf = np.copy(N_perf)

for i in range(size):
    # N = north, E = East
    #print (i)
    XXX = utm.from_latlon( perfil[1][i], perfil[0][i], force_zone_number=24 )
    N_perf[i], E_perf[i] = XXX[1], XXX[0] 

In [ ]:
 # ------------ plot the flight lines to check for the interpolation:--------#
# ------------ set figure size: ------------------------------------------#
plt.close('all')
fig = plt.figure(figsize=(12,4), facecolor='w')

plt.xlabel('Longitude ($^o$)', fontsize = 15)
plt.ylabel('Latitude ($^o$)', fontsize = 15)
p1 = plt.scatter(target['Lon'], target['Lat'], s=60.0, c=target['Boug_residual'], marker='.', cmap=plt.cm.jet, 
                vmin = target['Boug_residual'].min(), vmax = target['Boug_residual'].max(), alpha=0.3 )
#cbar = plt.colorbar(p1, aspect=30, fraction = 0.03, orientation="vertical")
#cbar.set_label('Residual Bouguer Anomaly $ (mGal) $',fontsize=15, rotation = 90)

# plot das linhas sismicas:
for i in range(0,2,1):
    ys[i] = [lat0[i], latf[i]]
    xs[i] = [lon0[i], lonf[i]]
    plt.plot(xs[i], ys[i], marker='.', linestyle='solid', linewidth=3.0, markersize=0, label=sis['LINHA'][i] )    

#plot dos pocos:
for k in range(0, len(poco), 1 ):
    i = ipoco[k] 
    plt.plot(poco['Lon'][i], poco['Lat'][i], marker='*',
             linestyle='None', linewidth=3.0, markersize=15, label=poco['Nome'][i] )
plt.legend()

# plot do perfil via funcao scatter:

p1 = plt.scatter(perfil[0], perfil[1], s=70.0, c=perfil[2], marker='.', cmap=plt.cm.jet, 
                 vmin = target['Boug_residual'].min(), vmax = target['Boug_residual'].max(), alpha=1.5)
cbar = plt.colorbar(p1, aspect=30, fraction = 0.03, orientation="vertical")
cbar.set_label('Anomalia Bouguer Residual $ (mGal) $',fontsize=15, rotation = 90)
plt.legend(loc=1)

###########################################################################################################################
#plt.savefig('figs finais/Python/mapa_perfil.png', dpi=300, transparent=True, bbox_inches='tight')

plt.show()

## 9.1: perfil do ajuste polinomial simples:

In [ ]:
# ------------ set figure size: ------------------------------------------#
plt.close('all')
fig = plt.figure(figsize=(18,8), facecolor='w')

plt.xlabel('UTM E ($m$)', fontsize = 15)
plt.ylabel('Anomalia Bouguer Residual ($mGal$)', fontsize = 15)


p1 = plt.scatter(E_perf, perfil[2], s=70.0, c=perfil[2], marker='.', cmap=plt.cm.jet,
                 vmin = target['Boug_residual'].min(), vmax = target['Boug_residual'].max())
cbar = plt.colorbar(p1, aspect=30, fraction = 0.03, orientation="vertical")
cbar.set_label('Anomalia Bouguer Residual $ (mGal) $',fontsize=15, rotation = 90)
plt.legend(loc=1)
plt.grid()
#plt.savefig('figs finais/Python/perfilrobusto.png', dpi=300, transparent=True, bbox_inches='tight')
plt.show()

## 9.2: perfil do ajuste polinomial robusto:

In [ ]:
# ------------ set figure size: ------------------------------------------#
plt.close('all')
fig = plt.figure(figsize=(18,8), facecolor='w')

plt.xlabel('UTM E ($m$)', fontsize = 15)
plt.ylabel('Anomalia Bouguer Residual ($mGal$)', fontsize = 15)


p1 = plt.scatter(E_perf, perfil_robusto[2], s=70.0, c=perfil_robusto[2], marker='.', cmap=plt.cm.jet,
                 vmin = target['Boug_residual_robusto'].min(), vmax = target['Boug_residual_robusto'].max())
cbar = plt.colorbar(p1, aspect=30, fraction = 0.03, orientation="vertical")
cbar.set_label('Anomalia Bouguer Residual $ (mGal) $',fontsize=15, rotation = 90)
plt.legend(loc=1)
plt.grid()
#plt.savefig('figs finais/Python/perfilrobusto.png', dpi=300, transparent=True, bbox_inches='tight')
plt.show()

# <center> Etapa 10: Mapa referenciado da Região de estudo via pacote Basemap : <center>

In [ ]:
# ------------------------- Define the center of the map -----------------------------------#  
lo0 = 0.5*( rec['Lon'].max() + rec['Lon'].min() )
la0 = 0.5*( rec['Lat'].max() + rec['Lat'].max() )

# ------- use basemap function to create the plot : ---------# 
m = Basemap(llcrnrlon=-39.0, llcrnrlat=-13.0,
            urcrnrlon=-38.0, urcrnrlat=-12.0, 
            resolution='l',projection='tmerc', 
            lon_0=lon0, lat_0=lat0,epsg=29100)

In [ ]:
# referenciando as coordenadas grav, sismica e poco ao mapa:
lons, lats = m( np.array( rec['Lon'] ), np.array( rec['Lat']) )

In [ ]:
# ------------ set figure size: ------------------------------------------#
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(18,15), facecolor='w')

#----------Drawing Lat Long: -------------#
m.drawmeridians( range( 0,360,1), labels=[0, 1, 0, 1], latmax=rec['Lat'].min() )
m.drawparallels( range(-90,90,1), labels=[1, 0, 1, 0], latmax=rec['Lat'].max() )
m.drawmapscale(-38.1, -12.85, 3, 3, 10.0, units='km', barstyle='simple', fontcolor='k')

#------------------Loading Shapefiles---------------#
#m.readshapefile('brazil_shapefile/BRA_adm0', 'Brasil', drawbounds=True, linewidth=1)
#m.readshapefile('Real/shapes/MatoGrossodoSul', 'MatoGrossodoSul', drawbounds=True, linewidth=1)
#m.readshapefile('Real/shapes/Parana', 'Parana', drawbounds=True, linewidth=1)
#m.readshapefile('shapeFiles/bacias/Limite_das_Bacias_Sedimentares', 'Limite_das_Bacias_Sedimentares', 
#                drawbounds=True, linewidth=1)



#--------------Background satellite image options-----------#

#m.shadedrelief(scale=1)
#m.etopo(scale=2)
#m.bluemarble(scale=0.5)
#m.arcgisimage(service='ESRI_Imagery_World_2D', xpixels=300, verbose=True)
m.arcgisimage(service='World_Shaded_Relief', xpixels=1000, ypixels=1000, verbose= True)


  
#---------- plot da linha sismica -------:
# plot das linhas sismicas:
for t in range(0,2,1):
    ys[t] = [lat0[t], latf[t]]
    xs[t] = [lon0[t], lonf[t]]
    m.plot(xs[t], ys[t], marker='.', linestyle='solid', linewidth=4.0,
           markersize=0, latlon=True, label=sis['LINHA'][t] ) 
    
#plot dos pocos:
for k in range(0, len(poco), 1 ):
    i = ipoco[k] 
    m.plot(poco['Lon'][i], poco['Lat'][i], marker='*', linestyle='None', linewidth=3.0,
           markersize=20.0, latlon=True, label=poco['Nome'][i] )   
plt.legend(loc=1)

#------------------Mapa da Anomalia------------------#
k = m.scatter(lons, lats, s=5.0, c=rec['Boug_residual'], marker='o', cmap=plt.cm.jet, latlon=False)
#----- colorbar specifications -----:
cbar = m.colorbar(k, location='right', size="2%", pad="2%")
cbar.set_label('Anomalia Bouguer Residual $ (mGal) $',fontsize=20, rotation=90)


#-------------Add Title-----------------------#
#plt.title('S ', fontsize=16, pad='10')
#plt.savefig('figs finais/Python/mapa_bacia_residual.png', dpi=300, transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
# Salvando arquivo para georeferenciamento no mapa geológico de reconcavo:
g = {'Lon': perfil[0], 'Lat': perfil[1], 'Boug_residual': perfil[2], 'UTMe': E_perf, 'UTMn': N_perf}
geo_ref = pd.DataFrame(g)

### TODO: verificar o por que o pandas coloca as colunas por ordem alfabética #####

# salvando arquivo para georeferenciamento for ROD:
geo_ref.to_csv('bouguer_recon_perfil.txt', sep = ' ', index=False)

## <center> Etapa 11: Visualização dos dados do modelo

In [ ]:
#Plot da função densidade do modelo

densidade= pd.read_csv('figs finais/dados finais/density7.txt', sep=' ', header=0 )

plt.close('all')

fig = plt.figure(figsize=(5,8))

# definition of fontsize:
fs = 15 
# invert axis
plt.gca().invert_yaxis()
plt.gca().invert_xaxis()

# set labelsize 
plt.tick_params(axis='y', labelsize=fs-2)
plt.tick_params(axis='x', labelsize=fs-2,labelbottom=True,labeltop=False)

plt.text(densidade['rhoZ'][0]+0.01, densidade['Z'][0]+0.148, str( format(densidade['rhoZ'][0],'.3f') ), style='italic',
        bbox={'facecolor':'red', 'alpha':0.5, 'pad':10})

plt.plot(densidade['rhoZ'],densidade['Z'],'k-',linewidth=2.0)
plt.plot(densidade['rhoZ'][0],densidade['Z'][0],'ro',linewidth=2.0)
plt.grid()
plt.xlabel(r'Contraste de densidade $(g/cm^3)$',fontsize=fs)
plt.ylabel(r'Profundidade $(m)$',fontsize=fs)
#plt.axis([np.max(rhoZ)+0.05, np.min(rhoZ)-0.02, np.max(Z)+0.1, np.min(Z)-0.1])
plt.savefig('figs finais/Python/funcao_densidade.png', dpi=300, transparent=True, bbox_inches="tight" )
plt.show()



In [ ]:
#leitura dos dados do modelo selecionado

modelo= pd.read_csv('figs finais/dados finais/model_basin_reconcavo5.txt', sep=' ', header=0 )

#descobrir numero de pontos do modelo

#print((modelo))

#plot do modelo selecionado
plt.close('all')
fig = plt.figure(figsize=(20,10), facecolor='w')

plt.xlabel('UTM E ($m$)', fontsize = 15)
plt.ylabel('profundidade ($m$)', fontsize = 15)
plt.plot(modelo['Easting(m)'], modelo['depth(m)'], 'k-')
plt.gca().invert_yaxis()

#plt.savefig('figs finais/Python/model_basin_reconcavo5.png', dpi=300, transparent=True, bbox_inches="tight" )
plt.show()


In [ ]:
#leitura dos dados do ajuste

#ajuste= pd.read_csv('figs finais/dados finais/datafit7.txt', sep=' ', header=0 )
plt.plot(ajuste['Easting(m)'], ajuste['Gobs(mGal)'],'.k', label='dado observado')
plt.plot(ajuste['Easting(m)'],ajuste['Gpred(mGal)'],'.r', label='dado predito')
plt.legend()
plt.grid()
plt.xlabel('UTM E (m)', fontsize = 14)
plt.ylabel('Anomalia Bouguer (mGal)', fontsize = 14)
#plt.savefig('figs finais/dados finais/datafit.png', dpi=300, transparent=True, bbox_inches="tight" )

plt.show()

